### This is text from the grant proposal I helped write. We can take/modify any of it for our project.

**Project Summary:** In a transportation system powered by liquid petroleum fuels, the majority of energy use and impacts are primarily caused by combustion emissions originating at individual vehicles. There may be uncertainty about the engine efficiency or the amount of specific pollutants released, but it is clear the emissions result from the combustion of liquid fuels and they are released at the same location as the vehicle. A transition to electric vehicles (EVs), will shift the energy source and emissions associated with transportation energy to power plants. At the same time, the grid is rapidly adding new and variable sources of power such as wind and solar. Identifying the source of electricity used to charge an EV is non-trivial, and different methods have been employed in recent work. Ideally, an analysis should identify and use the marginal generating unit (MGU) when estimating environmental emissions. The MGU at the time when an EV battery is charged might be any number of generators with vastly different emissions profiles, such as a renewable source, a newer natural gas combined cycle power plant, or an older coal-power plant. And in contrast to vehicles that use petroleum fuels, these emissions will be released at the power plant rather than at the vehicle. Changing the location of release does not change the impact of greenhouse gas (GHG) emissions, but it might change other environmental and human health impacts. 

Models that predict MGUs generally fall into one of two categories: regression-based or unit-commitment economic dispatch. The first category can account for effects (e.g. imperfect information) that are ignored in the second by examining real-world behavior. However, models that simply regress on historical behavior are ill-suited to making predictions about future grid conditions that differ from the data they were trained on. Applying machine learning techniques to energy sector analyses represents a pathway for large improvements in the understanding of energy transitions.

We propose to build a machine learning model that can predict the type of power plant that will provide marginal electricity in the state of Texas with a range of available fossil fuel power plants and renewable energy penetration. We will use this model to understand the implications of EV charging in the region. The model will use supervised learning algorithms to train on hourly generation and load data from 2007-2015, and will be designed so that it can meet the following goals:
1.	Examine how increased wind penetration affects the marginal power plant under otherwise identical net load (total load minus wind generation).
2.	Predict the type of MGU with grid conditions (e.g. wind penetration, operating fossil fuel power plants) that are different from those that exist today.
3.	Determine if marginal increases in generation come from the same power plants as marginal decreases in generation.
4.	Act as a modular piece of a larger model that can account for transmission constraints and trading between regions, and can be trained using data from other grid operators.

In order to accomplish these goals, we will focus on the following tasks:
- Task 1: Group all fossil fuel electric generating units in ERCOT using unsupervised classification (e.g. k-means), with inputs of fuel type and historical operating characteristics. Identify the group or groups of units that provides marginal generation at every hour over the 8 years of available data.
- Task 2: Set aside some part of the data for model validation. This validation set should have some characteristics (e.g. high penetration of wind power) that are not found in data used to train the model. 
- Task 3: Design and train the model on the remaining data using supervised learning algorithms.
- Task 4: Characterize the temporal and spatial energy and environmental implications of various levels of EV adoption in Texas.
